In [ ]:
!pip install pretty_midi mir_eval numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.6 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=e773c83319849225fd58d97866b3377fed092c25c2190331fd301c8820da5007
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100704 sha256=4846331c59b908599be5f4870d139350cfbd2758c1222cc7caf2e5be0ae44e21
  Stored in directory: /root/.cache/pip/wheels/3e/2f/0d/dda9c4c77a170e21356b6afa2f7d9bb078338634ba05d94e3f
Successfully built pretty_midi mir_eval
  Attempting uninstall: pack

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import subprocess

os.chdir("/content/drive/MyDrive/Dissertation Code/piano-vision")
!pip install -r requirements.txt

os.chdir("/content/drive/MyDrive/Dissertation Code/Skipping-The-Frame-Level")
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.4 

In [ ]:
import os
import pretty_midi
import mir_eval
import numpy as np
import itertools
import time
import subprocess

def read_note_data_from_text(file_path):
    intervals = []
    pitches = []
    with open(file_path, 'r') as file:
        for line in file:
            start, end, pitch = line.strip().split()
            intervals.append([float(start), float(end)])
            pitches.append(int(pitch))
    # print(np.array(intervals))
    return np.array(intervals), np.array(pitches)

def prepare_data_for_evaluation(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    intervals = []
    pitches = []
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            intervals.append([note.start, note.end])
            pitches.append(note.pitch)
    return np.array(intervals), np.array(pitches)

def evaluate_midi(predicted_midi_file, truth_midi_file):
    predicted_intervals, predicted_pitches = prepare_data_for_evaluation(predicted_midi_file)
    truth_intervals, truth_pitches = read_note_data_from_text(truth_midi_file)

    # mir_eval.transcription.validate(truth_intervals, truth_pitches, predicted_intervals, predicted_pitches)

    full_precision, full_recall, full_f1_score, _ = mir_eval.transcription.precision_recall_f1_overlap(
        truth_intervals, truth_pitches, predicted_intervals, predicted_pitches,
        onset_tolerance=0.05, pitch_tolerance=50.0, offset_ratio=None, offset_min_tolerance=0.05, strict=False, beta=1.0)

    onset_precision, onset_recall, onset_f1_score = mir_eval.transcription.onset_precision_recall_f1(
        truth_intervals, predicted_intervals, onset_tolerance=0.05, strict=False, beta=1.0)

    offset_precision, offset_recall, offset_f1_score = mir_eval.transcription.offset_precision_recall_f1(
        truth_intervals, predicted_intervals, offset_ratio=0.2, offset_min_tolerance=0.05, strict=False, beta=1.0)

    return {
        'full': (full_precision, full_recall, full_f1_score),
        'onset': (onset_precision, onset_recall, onset_f1_score),
        'offset': (offset_precision, offset_recall, offset_f1_score)
    }

def run_transcription_and_evaluate(input_video_path, output_midi_path, segment_size, segment_hop_size):
    command = f"python -m transkun.transcribe {input_video_path} {output_midi_path} --segmentHopSize {segment_hop_size} --segmentSize {segment_size} --device cuda"
    subprocess.run(command, check=True, shell=True)
    results = evaluate_midi(output_midi_path, truth_midi_path)
    full_f1_score = results['full'][2]
    return full_f1_score


In [ ]:
results = {}

os.chdir("/content/drive/MyDrive/Dissertation Code/Skipping-The-Frame-Level")

input_directory = "../OMAPS/complete/mp4-tuning"
output_directory = "../OMAPS/evaluation/audio-visual-fusion-tuning-4"
truth_folder = "../OMAPS/complete/text-tuning"

segment_sizes = [6, 8, 12, 16, 32, 64]
hop_size_ratios = [0.5, 0.75]

for segment_size, ratio in itertools.product(segment_sizes, hop_size_ratios):
    segment_hop_size = segment_size * ratio

    for filename in os.listdir(input_directory):
        if filename.endswith(".mp4"):
            base_name = os.path.splitext(filename)[0]
            modified_output_filename = f"{base_name}_seg{segment_size}_hop{segment_hop_size}.mid"
            input_video_path = os.path.join(input_directory, filename)
            output_midi_path = os.path.join(output_directory, modified_output_filename)
            truth_midi_path = os.path.join(truth_folder, base_name + ".txt")
            print(f"Processing with segment size {segment_size} and hop size {segment_hop_size}")

            start_time = time.time()
            try:
                f1_score = run_transcription_and_evaluate(input_video_path, output_midi_path, segment_size, segment_hop_size)
                processing_time = time.time() - start_time

                results[(segment_size, segment_hop_size)] = results.get((segment_size, segment_hop_size), []) + [f1_score]

                combination_result_path = os.path.join(output_directory, f"results_seg{segment_size}_hop{segment_hop_size}.txt")
                with open(combination_result_path, 'a') as combo_file:
                    combo_file.write(f"{filename}: F1 Score = {f1_score}, Processing Time = {processing_time} seconds\n")
            except subprocess.CalledProcessError as e:
                print(f"An error occurred: {e}")

average_f1_scores = {k: sum(v) / len(v) for k, v in results.items()}
best_parameters = max(average_f1_scores, key=average_f1_scores.get)
best_f1_score = average_f1_scores[best_parameters]

output_file_path = "/content/drive/MyDrive/Dissertation Code/OMAPS/evaluation/audio-visual-fusion-tuning-2/evaluation_results.txt"

with open(output_file_path, 'w') as file:
    file.write("Best Parameters:\n")
    file.write(f"Segment Size: {best_parameters[0]}, Hop Size: {best_parameters[1]}\n")
    file.write(f"Best F1 Score: {best_f1_score}\n")

print(f"Best Parameters: Segment Size = {best_parameters[0]}, Hop Size = {best_parameters[1]}")
print(f"Best Average F1 Score: {best_f1_score}")

Processing with segment size 6 and hop size 3.0...
Processing with segment size 6 and hop size 3.0...
Processing with segment size 6 and hop size 3.0...
Processing with segment size 6 and hop size 3.0...
Processing with segment size 6 and hop size 3.0...
Processing with segment size 6 and hop size 4.5...
Processing with segment size 6 and hop size 4.5...
Processing with segment size 6 and hop size 4.5...
Processing with segment size 6 and hop size 4.5...
Processing with segment size 6 and hop size 4.5...
Processing with segment size 8 and hop size 4.0...
Processing with segment size 8 and hop size 4.0...
Processing with segment size 8 and hop size 4.0...
Processing with segment size 8 and hop size 4.0...
Processing with segment size 8 and hop size 4.0...
Processing with segment size 8 and hop size 6.0...
Processing with segment size 8 and hop size 6.0...
Processing with segment size 8 and hop size 6.0...
Processing with segment size 8 and hop size 6.0...
Processing with segment size 8 

In [ ]:
!python -m transkun.transcribe ../OMAPS/complete/mp4-tuning/001.mp4 ../OMAPS/evaluation/audio-visual-fusion-tuning-2/001.mid --segmentHopSize 3 --segmentSize 3.5 --device cuda

error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evalu